Creating a Neural Network

In [67]:
pip install torch


Note: you may need to restart the kernel to use updated packages.


In [68]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [69]:
import pandas as pd

df = pd.read_csv('final_previous_merged.csv')

df = pd.get_dummies(df, columns=["prev_currency"], prefix="dummy", drop_first=False)

df.drop(columns={'prev_X.Other.Currency.to.USD', 'prev_inf_factor', 'prev_financialDate'}, inplace=True)
df.columns

Index(['prev_accountsPayable', 'prev_accountsReceivable',
       'prev_bankCashBalances', 'prev_bankOverdraft',
       'prev_debitOwnedWithinOneYear', 'prev_financialAssets',
       'prev_fixedAssets', 'prev_intangibleAssets', 'prev_otherCurrentAssets',
       'prev_otherCurrentLiabilities', 'prev_otherEquity',
       'prev_otherTermAssets', 'prev_otherTermLiabilities', 'prev_prepayments',
       'prev_retainedEarnings', 'prev_subscribedCapital', 'prev_termLoans',
       'prev_totalAssets', 'prev_totalCurrentAssets',
       'prev_totalCurrentLiabilities', 'prev_totalInventories',
       'prev_totalLiabilities', 'prev_totalShareholderEquity',
       'prev_totalTermAssets', 'prev_totalTermLiabilities',
       'prev_accountPayableAccrualFromCashFlow', 'prev_capitalExpenditure',
       'prev_changeInInventoriesFromCashFlow',
       'prev_netChangeInCashAndCashEquivalents', 'prev_netFundingCashFlow',
       'prev_netInvestingCashFlow',
       'prev_unbilledAccountsReceivableRevenueFromCashF

In [70]:
from sklearn.model_selection import train_test_split

# Drop the target column for X, keep 'FHR' as y
X = df.drop("FHR", axis=1).values  # shape: (num_samples, num_features)
y = df["FHR"].values               # shape: (num_samples,)

# Split into train and test sets
#   test_size=0.2 means 80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [71]:
# Convert NumPy arrays to Torch tensors (float)
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32)

X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.float32)

In [72]:
import torch.nn as nn

class FHRModel(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(FHRModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)  # hidden layer
        self.fc2 = nn.Linear(hidden_dim, 1)          # output layer

    def forward(self, x):
        # Simple forward with ReLU on hidden layer
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model
input_dim = X_train_t.shape[1]  # number of features
model = FHRModel(input_dim=input_dim, hidden_dim=20)


In [73]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [74]:
num_epochs = 7000

for epoch in range(num_epochs):
    # Set model to training mode (important if you have dropout or batchnorm)
    model.train()

    # Forward pass
    predictions = model(X_train_t).squeeze()  # shape: (train_size,)
    loss = criterion(predictions, y_train_t)

    # Backward pass & update
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print progress every few epochs
    if (epoch+1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


Epoch [10/7000], Loss: 140825.7031
Epoch [20/7000], Loss: 56494.7930
Epoch [30/7000], Loss: 30421.1719
Epoch [40/7000], Loss: 7241.1484
Epoch [50/7000], Loss: 4366.8164
Epoch [60/7000], Loss: 3450.3433
Epoch [70/7000], Loss: 2527.0803
Epoch [80/7000], Loss: 2038.8750
Epoch [90/7000], Loss: 1733.4932
Epoch [100/7000], Loss: 1527.1863
Epoch [110/7000], Loss: 1341.8009
Epoch [120/7000], Loss: 1202.8906
Epoch [130/7000], Loss: 1086.2822
Epoch [140/7000], Loss: 988.8872
Epoch [150/7000], Loss: 903.9279
Epoch [160/7000], Loss: 830.0682
Epoch [170/7000], Loss: 765.3193
Epoch [180/7000], Loss: 708.3344
Epoch [190/7000], Loss: 658.0068
Epoch [200/7000], Loss: 613.2224
Epoch [210/7000], Loss: 573.2981
Epoch [220/7000], Loss: 537.6122
Epoch [230/7000], Loss: 505.5313
Epoch [240/7000], Loss: 476.6058
Epoch [250/7000], Loss: 450.5085
Epoch [260/7000], Loss: 426.8938
Epoch [270/7000], Loss: 405.5860
Epoch [280/7000], Loss: 386.2012
Epoch [290/7000], Loss: 368.6416
Epoch [300/7000], Loss: 352.6750
Ep

In [75]:
# Set model to eval mode
model.eval()

with torch.no_grad():  # no gradient calculation needed
    test_predictions = model(X_test_t).squeeze()
    test_loss = criterion(test_predictions, y_test_t)

rmse = torch.sqrt(test_loss)
print(f"Test MSE: {test_loss.item():.4f}")
print(f"Test RMSE: {rmse.item():.4f}")


Test MSE: 111.9770
Test RMSE: 10.5819
